In [ ]:
import numpy as np
import sympy as sp
from General import functies as functies
from General import classes as classes
from matplotlib import pyplot as plt
from scipy.interpolate import CubicSpline

In [ ]:
#de exacte oplossing van die fucking derdegraadsveelterm die [H+] beschrijft
H, Ka, Na, c0, a = sp.symbols('H Ka Na c0 a', real = True) #a = concentratie H en OH door autoprotolyse van water
Kw = 10**(-7) #de waterconstante
""" #poging die fout bleek
vergelijking = sp.Eq(H**3 + Ka*H**2 + (Na-Kw*c0)*H + Ka*(Na - Kw), 0)
display(vergelijking)
oplossing = sp.solve(vergelijking, H)
for opl in oplossing:
    testval = opl.subs(Ka, 10**(-4.74)).subs(Na, 0).subs(c0, 1).evalf()
    display(testval)
    print('pH:')
    display(-np.log10(float(abs(testval))))
    print('_________')
    #skip dat imaginair deel, uit de 
print('____________take 2_______________')
#poging 2:"""
vergelijking = sp.Eq(Ka*(c0 + a)*(a+Na)**2 - Kw**2 + Kw*a*(a+Na), 0)
oplossing = sp.solve(vergelijking, a)
#testreeks 1 van de oplossingen:
"""
for opl in oplossing:
    H = Kw/(opl+Na)
    testval = H.subs(Ka, 10**(-4.74)).subs(Na, 0).subs(c0,1).evalf()
    display(testval)
    print('pH:')
    display(-np.log10(float(abs(testval))))
    print('______')
#gezien hier pH = 1/2 pKa - 1/2 log cA = 4.74/2, valt de eerste oplossing al af
print('Voor een 50/50 buffermengsel:')
for opl in oplossing:
    H = Kw/(opl+Na)
    testval = H.subs(Ka, 10**(-4.74)).subs(Na, 1/2).subs(c0,1).evalf()
    display(testval)
    print('pH:')
    display(-np.log10(float(abs(testval))))
    print('______')
#gezien hier pH = pKa - log(cA/cB) = 4.74 moet het de tweede oplossing (op index 1) zijn die we zoeken"""

exacteoplossing_a = sp.re(sp.simplify(oplossing[1].subs(Ka, 10**(-4.74))))
exacteoplossing = Kw/(exacteoplossing_a +Na)
display(exacteoplossing)
# Convert the symbolic expression to a numerical function
f_exacteoplossing = sp.lambdify((c0, Na), sp.re(exacteoplossing), 'numpy')


In [ ]:
def pH_buffer(pKa, ca, cb):
    return pKa - np.log10(ca/cb)

def nieuw_mengsel(ca, cb, deltaca, deltacb):
    canieuw = ca+deltaca-deltacb
    cbnieuw = cb-deltaca+deltacb
    return canieuw, cbnieuw

def pH_experiment_bufferzone(VNaOH, parameters):
    begin_n, pKa = parameters
    NaOH = VNaOH * 0.1 #Molair
    return pH_buffer(pKa, (begin_n - NaOH)/(40e-3 + VNaOH), NaOH/(40e-3 + VNaOH))

def pH_experiment_verzadigd(VNaOH, parameters):
    begin_n, pKa = parameters
    cCH3COOmin = begin_n/(40e-3+VNaOH)
    if type(VNaOH) != np.array:
        VNaOH = np.array([VNaOH])
    #nulde orde benadering
    nOH = []
    for volume in VNaOH:
        nOH.append(np.max([volume*0.1 - begin_n, 1e-7])) #neem aan dat het natriumacetaat wegreageert en niet meer relevant is; echter steeds meer OH- dan de waterconstante
    nOH = np.array(nOH)
    cOH = nOH/(40e-3 + VNaOH)

    #eerste orde correctie: neem aan dat de concentratie OH- constant is maar wordt bijgevuld door CH3COONa  + H2O-> CH3COOH + Na+ + OH-
    Kb = 10**(-(14-pKa))
    cOH += Kb * cCH3COOmin/cOH
    return 14 + np.log10(cOH)

def pH_experiment(VNaOH, parameters):
    c0val = parameters[0]
    concentratie_CH3COOH = c0val * 40e-3/(40e-3 + VNaOH)
    nNaOH = VNaOH*0.1 #stofhoeveelheid natriumhydroxide
    cNaOH = nNaOH/(40e-3+VNaOH)
    output = sp.lambdify([c0, Na], sp.re(exacteoplossing), 'numpy')(concentratie_CH3COOH, cNaOH) #deze shit is kapot
    """
    output = []
    for beginc in concentratie_CH3COOH:
        nieuw = []
        for basec in cNaOH:
            nieuw.append(abs(exacteoplossing.subs(c0, beginc).subs(Na, basec)))
        output.append(np.array(nieuw))
    output = np.array(output)
    """
    return output

def pH_experiment_approc(VNaOH, parameters):
    begin_n, pKa = parameters
    c0val = begin_n/(40e-3)
    output = []
    for vol in VNaOH:
        if 40e-3*c0val > 0.1*vol: #buffergebied
            output.append(pH_experiment_bufferzone(vol, parameters))
        else:
            output.append(pH_experiment_verzadigd(vol, parameters)[0])
    return np.array(output)

In [ ]:
#voorbereidende oefening
cabegin = 0.5 #alles in molair doen
cbbegin = 0.5
pKa = 4.76
extrazuur = np.linspace(-0.5,0.5) #negatief toegevoegd zuur = toegevoegd base
fig, ax = plt.subplots(1,1)
ax.plot(extrazuur, pH_buffer(pKa, *nieuw_mengsel(cabegin, cbbegin, extrazuur, 0)), label = 'pH')
D1 = pH_buffer(pKa, *nieuw_mengsel(cabegin, cbbegin, extrazuur[:-1], 0))
D2 = pH_buffer(pKa, *nieuw_mengsel(cabegin, cbbegin, extrazuur[1:], 0))
afgeleide = (D2-D1)/(extrazuur[1]-extrazuur[0])
ax.plot(extrazuur[1:], afgeleide, label = 'afgeleide van de pH naar de concentratie zuur')
ax.set_xlabel('extra toegevoegd zuur (positief) of base (negatief) [M]')
ax.set_ylabel('pH/mol')
ax.legend()
plt.tight_layout()
plt.show()
print('minimum van de afgeleide:')
print('x = ',(extrazuur[np.argmax(afgeleide)] + extrazuur[1+np.argmax(afgeleide)])/2, ' dy/dx = ', np.max(afgeleide))
#het is duidelijk minimaal rond 0

In [ ]:
#de plot van de data en zo
adres = 'buffer/titratie2.txt'
volumes,pHs = np.loadtxt(adres, delimiter=",").T #volumes in ml
volumes *= 10**(-3) #vanaf nu volumes in l
volumelinspace = np.linspace(0.1e-3,20e-3,500000)
pHfunctie = CubicSpline(volumes, pHs)

#maak die rare constructie
#creeer de raaklijnen als lambdafuncties
afgeleide_pHfunctie = pHfunctie.derivative(nu = 1)
tweedeafgeleide_pHfunctie = pHfunctie.derivative(nu=2)
deriv2 = tweedeafgeleide_pHfunctie(volumelinspace)
minderiv2indx = np.argmin(deriv2)
maxderiv2indx = np.argmax(deriv2)
x1 = volumelinspace[minderiv2indx]
x2 = volumelinspace[maxderiv2indx]
rico1 = afgeleide_pHfunctie(x1)
rico2 = afgeleide_pHfunctie(x2)
rico = (rico1 + rico2)/2
y1 = pHfunctie(x1)
y2 = pHfunctie(x2)
raaklijn1 = lambda x: rico*(x-x1) + y1
raaklijn2 = lambda x: rico*(x-x2) + y2
print(rico)
#creeer die rare loodlijn daar
algemeneloodlijn = lambda x,b: -1/rico *x + b
print(-1/rico)
bwaardes = np.linspace(0,14,10000)
oppervlakteverschil = 1e6
for bval in bwaardes:
    snijpunt1 = (bval - y1 + rico*x1)/(rico + 1/rico) #snijpunt tussen raaklijn 1 en de loodlijn
    snijpunt2 = (bval - y2 + rico*x2)/(rico + 1/rico) #snijpunt tussen raaklijn 2 en de loodlijn
    verschil1 = lambda x: pHfunctie(x) - raaklijn1(x)
    verschil2 = lambda x: raaklijn2(x) - pHfunctie(x)
    xrange1 = np.linspace(x1, snijpunt1, 10000)
    xrange2 = np.linspace(snijpunt2, x2, 10000)
    oppervlakte1 = np.trapz(verschil1(xrange1), xrange1)
    oppervlakte2 = np.trapz(verschil2(xrange2), xrange2)
    delta = abs(oppervlakte1 - oppervlakte2)
    if delta < oppervlakteverschil:
        oppervlakteverschil = delta
        B = bval
print(B)
loodlijn = lambda x: algemeneloodlijn(x,B)
#zoek de intersectie tussen de loodlijn en de pHfunctie
afstand = 1e6
for volume in volumelinspace:
    delta = abs(loodlijn(volume) - pHfunctie(volume))
    if delta < afstand:
        afstand = delta
        equivalent_volume = volume

print('Het equivalent volume is gelijk aan', equivalent_volume, 'L')


fig, ax = plt.subplots(1,1, figsize = (10,10))
ax.errorbar(volumes, pHs, xerr = 0.5e-3, yerr = 0.01, linestyle = '', label = 'data')
ax.plot(volumelinspace, pHfunctie(volumelinspace), label = 'interpolatie')
ax.plot(volumelinspace, pH_experiment_approc(volumelinspace, [0.001, 4.74]), label = 'model', zorder = 2)
ax.plot(volumelinspace, raaklijn1(volumelinspace), linestyle = '--', label = 'raaklijn 1')
ax.plot(volumelinspace, raaklijn2(volumelinspace), linestyle = '--', label = 'raaklijn 2')
ax.plot(volumelinspace, loodlijn(volumelinspace), linestyle = '--', label = 'loodlijn')
ax.set_ylabel('pH')
ax.set_xlabel('Volume toegevoegd NaOH [l]')
ax.legend()
ax.set_ylim(0,14)
#ax.set_xlim(0.0075, 0.0125)
plt.tight_layout()
plt.show()

In [ ]:
#bereken de molariteit in de maatkolf en zo
Veq, Vsamp, Vkolf, ckolf, M, mpercent = sp.symbols('Veq Vsamp Vkolf ckolf M m%')
volume_maatkolf = classes.vergelijking(Veq*0.1/Vsamp, [Veq, Vsamp]) #Veq: equivalent volume, Vsamp: volume sample uit maatkolf
equivol = classes.datapunt(equivalent_volume, 1e-3, Veq)#neem aan: fout op equivalent volume is 1ml
sampvol = classes.datapunt(10e-3, 0.02e-3, Vsamp) #fout op samplevolume is 0.02ml, zie cursus maatglaswerk
molariteit_maatkolf = volume_maatkolf.evaluate([equivol, sampvol], None, ckolf)
print('concentratie in de maatkolf (in molair) is gelijk aan: ')
functies.latex_print_datapunt(molariteit_maatkolf)

massapercent_winkelazijn = classes.vergelijking(ckolf*Vkolf/Vsamp * M  * 1/1000*100, [ckolf, Vkolf, Vsamp, M]) #de *1/1000 is om van g/L = g/kg naar g/g te gaan, de *100 maakt er een percentage van
kolfvolume = classes.datapunt(100e-3, 0.1e-3, Vkolf)
molairemassa = classes.datapunt(60.0, 0.1, M)
massapercent = massapercent_winkelazijn.evaluate([kolfvolume, molairemassa, sampvol, molariteit_maatkolf], None, mpercent)
print('het massapercent azijn in de winkelazijn is gelijk aan: ')
functies.latex_print_datapunt(massapercent)


In [ ]:
#fit overal:
def invals(x,y):
    return [0.001,4.74]
functies.fit_2D(['n0', 'pKa'], pH_experiment_approc, invals, volumes, pHs, np.full(np.shape(volumes),1e-3)**2/12, np.full(np.shape(pHs),0.01)**2/12,  x_as_titel= 'toegevoegd volume (l)', y_as_titel = 'pH', bounds=[(1e-7,1), (4,7)], detailed_logs=True)

In [ ]:
#test het met 1D fit:
outp = functies.fit(['n0', 'pKa'], pH_experiment_approc, invals, volumes, pHs, np.full(np.shape(pHs),0.01))#,  x_as_titel= 'toegevoegd volume (l)', y_as_titel = 'pH', bounds=[(0,5), (4,7)], detailed_logs=False)

In [ ]:
#fit het buffergebied binnen pKa - 1 tot pKa + 1
indxmin = 0
while pHs[indxmin] < 3.74:
    indxmin+=1
indxmax = indxmin+1
while pHs[indxmax] < 5.74:
    indxmax += 1
volumesreduced = volumes[indxmin: indxmax]
pHsreduced = pHs[indxmin: indxmax]
def invals(x,y):
    return [0.001, 4.69]
functies.fit_2D(['n_0', 'pKa'], pH_experiment_bufferzone, invals, volumesreduced, pHsreduced, np.full(np.shape(volumesreduced),1e-3)**2/12, np.full(np.shape(pHsreduced),0.01)**2/12,
                 detailed_logs=True, x_as_titel= 'toegevoegd volume (l)', y_as_titel = 'pH', bounds=[(0.00094,5), (0,14)])